In [ ]:
from collections import defaultdict
from typing import List
from collections import deque

In [ ]:
# https://leetcode.com/problems/number-of-islands/description/
class Solution:
    def numIslands(self, grid: List[List[str]]) -> int:
        m, n = len(grid), len(grid[0])
        no_islands = 0
        # def dfs(i ,j):
        #     if i<0 or i>=m or j<0 or j>=n or grid[i][j] != '1':
        #         return
        #     else:
        #         grid[i][j] = '0'
        #         dfs(i, j+1)
        #         dfs(i+1, j)
        #         dfs(i, j-1)
        #         dfs(i-1, j)

        # for i in range(m):
        #     for j in range(n):
        #         if grid[i][j] == '1':
        #             no_islands +=1
        #             dfs(i,j)
        # return no_islands
        for i in range(m):
            for j in range(n):
                if grid[i][j] == '1':
                    no_islands +=1
                    grid[i][j] = '0'
                    neighbors = deque()
                    neighbors.append((i, j))
                    while neighbors:
                        r, c = neighbors.popleft()
                        if r-1 >= 0 and grid[r-1][c] == '1':
                            neighbors.append((r-1, c))
                            grid[r-1][c] = '0'
                        if r+1 < m and grid[r+1][c] == '1':
                            neighbors.append((r+1, c))
                            grid[r+1][c] = '0'
                        if c-1 >= 0 and grid[r][c-1] == '1':
                            neighbors.append((r, c-1))
                            grid[r][c-1] = '0'
                        if c+1 < n and grid[r][c+1] == '1':
                            neighbors.append((r, c+1))
                            grid[r][c+1] = '0'
        return no_islands

In [ ]:
# https://leetcode.com/problems/pacific-atlantic-water-flow/description/
class Solution:
    def pacificAtlantic(self, heights):
        if not heights or not heights[0]:
            return []

        ROWS, COLS = len(heights), len(heights[0])
        pacific = set()
        atlantic = set()

        def dfs(r, c, visited):
            visited.add((r, c))
            for dr, dc in [(1,0), (-1,0), (0,1), (0,-1)]:
                nr, nc = r + dr, c + dc
                if (
                    0 <= nr < ROWS and
                    0 <= nc < COLS and
                    (nr, nc) not in visited and
                    heights[nr][nc] >= heights[r][c]
                ):
                    dfs(nr, nc, visited)

        # Pacific: top row + left column
        for c in range(COLS):
            dfs(0, c, pacific)
        for r in range(ROWS):
            dfs(r, 0, pacific)

        # Atlantic: bottom row + right column
        for c in range(COLS):
            dfs(ROWS - 1, c, atlantic)
        for r in range(ROWS):
            dfs(r, COLS - 1, atlantic)

        # Cells reachable by both
        return list(pacific & atlantic)


In [ ]:
# https://leetcode.com/problems/course-schedule/description/
class Solution:
    def canFinish(self, n: int, edges: List[List[int]]) -> bool:
        adj = [[] for _ in range(n)]
        # # Indegree is number of incoming edges to a node
        # indegree = [0] * n
        # # build graph
        # for u, v in edges:
        #     adj[u].append(v)
        #     indegree[v] += 1
        # # queue of nodes with indegree 0
        # q = deque()
        # for i in range(n):
        #     if indegree[i] == 0:
        #         q.append(i)
        # topo = []
        # while q:
        #     node = q.popleft()
        #     topo.append(node)
        #     for nei in adj[node]:
        #         indegree[nei] -= 1
        #         if indegree[nei] == 0:
        #             q.append(nei)
        # # cycle check
        # if len(topo) != n:
        #     return False
        # return True # or directly we can return len(topo) == n
        
        for u, v in edges:
            adj[u].append(v)
        visit = [False] * n      # fully processed nodes
        inStack = [False] * n     # nodes in current DFS path
        def dfs(node):
            # If the node is already in the stack, we have a cycle.
            if inStack[node]:
                return True
            if visit[node]:
                return False
            # Mark the current node as visited and part of current recursion stack.
            visit[node] = True
            inStack[node] = True
            for neighbor in adj[node]:
                if dfs(neighbor):
                    return True
            # Remove the node from the stack.
            inStack[node] = False
            return False
        # Run DFS from every node (handles disconnected graph)
        for i in range(n):
            if dfs(i):
                return False
        return True


In [ ]:
# https://leetcode.com/problems/course-schedule-ii/description/
class Solution:
    def findOrder(self, numCourses: int, prerequisites: List[List[int]]) -> List[int]:
        adj = [[] for i in range(numCourses)]
        for u,v in prerequisites:
            adj[v].append(u)
        visited = [False]*numCourses
        visiting = [False]*numCourses
        topo = []
        def dfs(node):
            if visiting[node]:
                return False
            if visited[node]:
                return True
            visiting[node] = True
            for nei in adj[node]:
                if not dfs(nei):
                    return False
            visiting[node] = False
            visited[node] = True
            topo.append(node)
            return True
        for i in range(numCourses):
            if not visited[i]:
                if not dfs(i):
                    return []
        return topo[::-1]


In [ ]:
# https://leetcode.com/problems/graph-valid-tree/description/
class Solution:
    def validTree(self, n: int, edges: List[List[int]]) -> bool:
        # Rule 1: tree must have exactly n - 1 edges
        if len(edges) != n - 1:
            return False
        parent = [i for i in range(n)]
        def find(x):
            if parent[x] != x:
                parent[x] = find(parent[x])  # path compression
            return parent[x]

        def union(x, y):
            px, py = find(x), find(y)
            if px == py:
                return False  # cycle detected
            parent[py] = px
            return True
        for u, v in edges:
            if not union(u, v):
                return False
        return True

In [ ]:
# https://leetcode.com/problems/rotting-oranges/description/
def orangesRotting(self, grid: List[List[int]]) -> int:
        minutes  = 0
        m, n = len(grid), len(grid[0])
        directions = [(0,1), (1,0), (-1,0), (0,-1)]
        fresh = 0
        q = deque()
        for i in range(m):
            for j in range(n):
                if grid[i][j] == 2:
                    q.append((i,j))
                elif grid[i][j] == 1:
                    fresh +=1
        while q and fresh >0:
            minutes +=1
            for _ in range(len(q)):
                r, c = q.popleft()
                for dr, dc in directions:
                    nr, nc = r+dr, c+dc
                    if 0<=nr<m and 0<=nc<n and grid[nr][nc]==1:
                        q.append((nr,nc))
                        grid[nr][nc] = 2
                        fresh -=1
        return minutes if fresh == 0 else -1